In [58]:
import pandas as pd
import numpy as np

In [50]:
joe = pd.read_csv('hashtag_joebiden.csv', lineterminator='\n', parse_dates=True)
trump = pd.read_csv('hashtag_donaldtrump.csv', lineterminator='\n', parse_dates=True)

I'll start by combining the two large batches of tweets with #Trump or #Biden tweets.

In [52]:
combined_df = pd.concat([joe, trump], ignore_index=True)

In [54]:
combined_df.head()

,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,...,user_followers_count,user_location,lat,long,city,country,continent,state,state_code,collected_at
0,2020-10-15 00:00:01,1.316529e+18,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,TweetDeck,3.606665e+08,El Sol Latino News,elsollatinonews,🌐 Noticias de interés para latinos de la costa...,...,1860.0,"Philadelphia, PA / Miami, FL",25.774270,-80.193660,NaN,United States of America,North America,Florida,FL,2020-10-21 00:00:00
1,2020-10-15 00:00:18,1.316529e+18,#HunterBiden #HunterBidenEmails #JoeBiden #Joe...,0.0,0.0,Twitter for iPad,8.099044e+08,Cheri A. 🇺🇸,Biloximeemaw,"Locked and loaded Meemaw. Love God, my family ...",...,6628.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-21 00:00:00.517827283
2,2020-10-15 00:00:20,1.316529e+18,@IslandGirlPRV @BradBeauregardJ @MeidasTouch T...,0.0,0.0,Twitter Web App,3.494182e+09,Flag Waver,Flag_Wavers,NaN,...,1536.0,Golden Valley Arizona,46.304036,-109.171431,NaN,United States of America,North America,Montana,MT,2020-10-21 00:00:01.035654566
3,2020-10-15 00:00:21,1.316529e+18,@chrislongview Watching and setting dvr. Let’s...,0.0,0.0,Twitter for iPhone,8.242596e+17,Michelle Ferg,MichelleFerg4,NaN,...,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-21 00:00:01.553481849
4,2020-10-15 00:00:22,1.316529e+18,#censorship #HunterBiden #Biden #BidenEmails #...,1.0,0.0,Twitter Web App,1.032807e+18,the Gold State,theegoldstate,A Silicon Valley #independent #News #Media #St...,...,390.0,"California, USA",36.701463,-118.755997,NaN,United States of America,North America,California,CA,2020-10-21 00:00:02.071309132


In [53]:
combined_df.isnull().sum()

created_at                    0
tweet_id                      0
tweet                         0
likes                         0
retweet_count                 0
source                     1425
user_id                       0
user_name                    45
user_screen_name              0
user_description         148550
user_join_date                0
user_followers_count          0
user_location            432777
lat                      764445
long                     764445
city                    1087420
country                  767855
continent                767825
state                    943588
state_code               972252
collected_at                  1
dtype: int64

In [55]:
combined_df.nunique()

created_at               817782
tweet_id                1287607
tweet                   1273922
likes                      2023
retweet_count               868
source                     1005
user_id                  420376
user_name                393419
user_screen_name         421095
user_description         370670
user_join_date           420002
user_followers_count      46458
user_location            100769
lat                        6537
long                       6539
city                       1507
country                     189
continent                     7
state                       710
state_code                  308
collected_at            1429943
dtype: int64

Looks like we're missing a lot of location data.

In [57]:
combined_df.dtypes

created_at               object
tweet_id                float64
tweet                    object
likes                   float64
retweet_count           float64
source                   object
user_id                 float64
user_name                object
user_screen_name         object
user_description         object
user_join_date           object
user_followers_count    float64
user_location            object
lat                     float64
long                    float64
city                     object
country                  object
continent                object
state                    object
state_code               object
collected_at             object
dtype: object

For each user, I determine their state, city, country by the mode of each. Their number of followers is the maximal number of followers they had across the dataset.

In [98]:
combined_df['user_followers_count'] = pd.to_numeric(combined_df['user_followers_count'], errors='coerce')

def agg_mode(series):
    modes = series.mode()
    return modes.iloc[0] if not modes.empty else None

result_df = combined_df.groupby(['user_id', 'user_screen_name']).agg({
    'user_followers_count': 'max',
    'state': agg_mode,
    'city': agg_mode,
    'country': agg_mode
}).reset_index()

result_df.rename(columns={'user_screen_name': 'handle'}, inplace=True)
result_df['is_politician'] = np.nan
result_df['political_affiliation'] = np.nan

In [99]:
result_df

,user_id,handle,user_followers_count,state,city,country,is_politician,political_affiliation
0,5.310000e+02,DocRob,5853.0,New York,None,United States of America,NaN,NaN
1,1.081000e+03,davemcclure,357619.0,California,Cupertino,United States of America,NaN,NaN
2,1.889000e+03,JonAshley,322.0,None,None,None,NaN,NaN
3,2.654000e+03,data4all,6342.0,Massachusetts,None,United States of America,NaN,NaN
4,2.737000e+03,jotbe,693.0,None,None,Germany,NaN,NaN
...,...,...,...,...,...,...,...,...
421095,1.325551e+18,HaddushHana,5.0,None,None,None,NaN,NaN
421096,1.325564e+18,Lenniox1,2.0,None,None,None,NaN,NaN
421097,1.325564e+18,realWarroom,886.0,None,None,None,NaN,NaN
421098,1.325568e+18,Notreal91325543,4.0,None,None,None,NaN,NaN


In [100]:
pol_tweets = pd.read_csv('ExtractedTweets.csv')

In [101]:
pol_tweets.nunique()

Party         2
Handle      433
Tweet     84502
dtype: int64

Here, I had to manually insert what state each of the representatives was from, if applicable - there was no readily apparent easier solution to this.

In [ ]:
def map_handles_to_states(df):
    unique_handles = df['Handle'].unique()
    handle_state_map = {}
    for handle in unique_handles:
        state = input(f"Enter the state for {handle}: ")
        handle_state_map[handle] = state

    return handle_state_map
handle_state_map = map_handles_to_states(pol_tweets)
print(handle_state_map)

Here, I isolate unique handles and apply the manual state mapping I input directly above, and construct the df to match with user information from the other dataset as much as possible.

In [138]:
unique_handles_df = pol_tweets[['Handle', 'Party']].drop_duplicates().reset_index(drop=True)
unique_handles_df['State'] = unique_handles_df['Handle'].map(handle_state_map).replace('', np.nan)

pol_df = pd.DataFrame({
    'user_id': np.nan,
    'handle': unique_handles_df['Handle'],
    'user_followers_count': np.nan,
    'state': unique_handles_df['State'],
    'city': np.nan,
    'country': 'United States of America',
    'is_politician': 'Yes',
    'political_affiliation': unique_handles_df['Party']
})

pol_df['state'] = pol_df['state'].replace('', np.nan).fillna(np.nan)

In [139]:
pol_df

,user_id,handle,user_followers_count,state,city,country,is_politician,political_affiliation
0,NaN,RepDarrenSoto,NaN,Florida,NaN,United States of America,Yes,Democrat
1,NaN,RepJackyRosen,NaN,Nevada,NaN,United States of America,Yes,Democrat
2,NaN,RepAlLawsonJr,NaN,Florida,NaN,United States of America,Yes,Democrat
3,NaN,RepEspaillat,NaN,New York,NaN,United States of America,Yes,Democrat
4,NaN,RepBRochester,NaN,Delaware,NaN,United States of America,Yes,Democrat
...,...,...,...,...,...,...,...,...
428,NaN,RobWittman,NaN,Virginia,NaN,United States of America,Yes,Republican
429,NaN,RosLehtinen,NaN,Florida,NaN,United States of America,Yes,Republican
430,NaN,WaysandMeansGOP,NaN,NaN,NaN,United States of America,Yes,Republican
431,NaN,GOPpolicy,NaN,NaN,NaN,United States of America,Yes,Republican


Fixing up some none values to NaN.

In [105]:
result_df.rename(columns={'user_screen_name': 'user_id'}, inplace=True)
result_df['is_politician'] = 'No'
for column in ['city', 'state', 'country']:
    result_df[column] = result_df[column].replace({None: np.nan})

In [119]:
result_df

,user_id,handle,user_followers_count,state,city,country,is_politician,political_affiliation
0,5.310000e+02,DocRob,5853.0,New York,NaN,United States of America,No,NaN
1,1.081000e+03,davemcclure,357619.0,California,Cupertino,United States of America,No,NaN
2,1.889000e+03,JonAshley,322.0,NaN,NaN,NaN,No,NaN
3,2.654000e+03,data4all,6342.0,Massachusetts,NaN,United States of America,No,NaN
4,2.737000e+03,jotbe,693.0,NaN,NaN,Germany,No,NaN
...,...,...,...,...,...,...,...,...
421095,1.325551e+18,HaddushHana,5.0,NaN,NaN,NaN,No,NaN
421096,1.325564e+18,Lenniox1,2.0,NaN,NaN,NaN,No,NaN
421097,1.325564e+18,realWarroom,886.0,NaN,NaN,NaN,No,NaN
421098,1.325568e+18,Notreal91325543,4.0,NaN,NaN,NaN,No,NaN


In [124]:
min(result_df['user_id'])

531.0

The minimum user_id is greater than the number of unique politician ID's; this is useful, as I can create a placeholder user_id that is unique equal to their index in a politician pool only, except in the below case:

Here, I assign the user_id associated with the handle in the larger dataset if applicable (i.e. the 2018 representative happened to tweet during the two week time span, revealing this information)..

In [142]:
mapped_ids = pol_df['handle'].map(result_df_unique.set_index('handle')['user_id'])
pol_df['user_id'] = mapped_ids
final_df['user_id'] = pol_df['user_id'].combine_first(pd.Series(pol_df.index, index=pol_df.index))

In [144]:
pol_df

,user_id,handle,user_followers_count,state,city,country,is_politician,political_affiliation
0,0.0,RepDarrenSoto,NaN,Florida,NaN,United States of America,Yes,Democrat
1,1.0,RepJackyRosen,NaN,Nevada,NaN,United States of America,Yes,Democrat
2,2.0,RepAlLawsonJr,NaN,Florida,NaN,United States of America,Yes,Democrat
3,3.0,RepEspaillat,NaN,New York,NaN,United States of America,Yes,Democrat
4,4.0,RepBRochester,NaN,Delaware,NaN,United States of America,Yes,Democrat
...,...,...,...,...,...,...,...,...
428,428.0,RobWittman,NaN,Virginia,NaN,United States of America,Yes,Republican
429,429.0,RosLehtinen,NaN,Florida,NaN,United States of America,Yes,Republican
430,430.0,WaysandMeansGOP,NaN,NaN,NaN,United States of America,Yes,Republican
431,431.0,GOPpolicy,NaN,NaN,NaN,United States of America,Yes,Republican


In [145]:
final_concatenated_df = pd.concat([pol_df, result_df], ignore_index=True)

In [146]:
final_concatenated_df

,user_id,handle,user_followers_count,state,city,country,is_politician,political_affiliation
0,0.000000e+00,RepDarrenSoto,NaN,Florida,NaN,United States of America,Yes,Democrat
1,1.000000e+00,RepJackyRosen,NaN,Nevada,NaN,United States of America,Yes,Democrat
2,2.000000e+00,RepAlLawsonJr,NaN,Florida,NaN,United States of America,Yes,Democrat
3,3.000000e+00,RepEspaillat,NaN,New York,NaN,United States of America,Yes,Democrat
4,4.000000e+00,RepBRochester,NaN,Delaware,NaN,United States of America,Yes,Democrat
...,...,...,...,...,...,...,...,...
421528,1.325551e+18,HaddushHana,5.0,NaN,NaN,NaN,No,NaN
421529,1.325564e+18,Lenniox1,2.0,NaN,NaN,NaN,No,NaN
421530,1.325564e+18,realWarroom,886.0,NaN,NaN,NaN,No,NaN
421531,1.325568e+18,Notreal91325543,4.0,NaN,NaN,NaN,No,NaN


Depending on how one approaches the data, you may want to restrict to one entry per user id: this is how I would proceed.

In [147]:
final_concatenated_df = final_concatenated_df.sort_values(by=['is_politician', 'user_followers_count'], ascending=False)
final_concatenated_df = final_concatenated_df.drop_duplicates(subset='user_id', keep='first')
final_concatenated_df.reset_index(drop=True, inplace=True)

In [148]:
final_concatenated_df

,user_id,handle,user_followers_count,state,city,country,is_politician,political_affiliation
0,0.000000e+00,RepDarrenSoto,NaN,Florida,NaN,United States of America,Yes,Democrat
1,1.000000e+00,RepJackyRosen,NaN,Nevada,NaN,United States of America,Yes,Democrat
2,2.000000e+00,RepAlLawsonJr,NaN,Florida,NaN,United States of America,Yes,Democrat
3,3.000000e+00,RepEspaillat,NaN,New York,NaN,United States of America,Yes,Democrat
4,4.000000e+00,RepBRochester,NaN,Delaware,NaN,United States of America,Yes,Democrat
...,...,...,...,...,...,...,...,...
420783,1.325539e+18,GenZstuff,0.0,NaN,NaN,NaN,No,NaN
420784,1.325544e+18,Sheena14785566,0.0,NaN,NaN,NaN,No,NaN
420785,1.325545e+18,LindsayHathawa8,0.0,NaN,NaN,NaN,No,NaN
420786,1.325545e+18,IndieWonksUSA,0.0,NaN,NaN,NaN,No,NaN


In [137]:
final_concatenated_df.to_csv('users.csv', index=False)